In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime

from summer.utils import ref_times_to_dti

from autumn.settings.constants import GOOGLE_MOBILITY_LOCATIONS
from autumn.core.inputs.mobility.queries import get_mobility_data
from autumn.models.sm_sir.mixing_matrix.macrodistancing import weight_mobility_data
from autumn.core import inputs
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.utils.display import pretty_print
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes
from autumn.core.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline

Grab a few reused parameters

In [ ]:
country = baseline_params["country"]["iso3"]
sub_region = baseline_params["population"]["region"]
data_year = baseline_params["population"]["year"]
ref_matrix_country = baseline_params["ref_mixing_iso3"]

### Population

In [ ]:
modelled_age_groups = baseline_params["age_groups"]
age_pops = pd.Series(
    inputs.get_population_by_agegroup(
        modelled_age_groups,
        country,
        sub_region,
        data_year,
    ),
    index=modelled_age_groups
)
print(f"Total modelled population is {sum(age_pops) / 1e3} thousand")
print(f"Modelled age groups are {modelled_age_groups}")
age_pops.plot.bar()

### Mixing matrix

In [ ]:
print(f"Target country: {country}")
print(f"Target sub-region: {sub_region}")
print(f"Proxy country: {ref_matrix_country}")
print("Always age-adjusted to target population under current SM-SIR code")

Note that in the Victoria Nature Communications paper, we used the UK Polymod matrices in our base analysis as the reference matrices, and the Belgium Polymod matrices for our sensitivity analysis.

In [ ]:
mixing_matrices = build_synthetic_matrices(
    country,
    ref_matrix_country,
    [int(age) for age in modelled_age_groups],
    True,
    sub_region,
)

In [ ]:
print(f"Total daily contacts for each age group is {mixing_matrices['all_locations'].sum(axis=1)}")
px.imshow(mixing_matrices["all_locations"])

### Mobility

In [ ]:
mob_df, google_mobility_days = get_mobility_data(
    country, 
    sub_region,
    COVID_BASE_DATETIME,
)
mob_dates_df = mob_df.copy()
mob_dates_df.index = mob_df["date"]

In [ ]:
mob_dates_df[GOOGLE_MOBILITY_LOCATIONS].plot()

### Calibration targets

In [ ]:
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [ ]:
ts_set_dates["notifications"].plot()